In [279]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [280]:
from deluca.agents import BPC, LQR
from deluca.envs import LDS
import jax.numpy as jnp

In [281]:
A,B = jnp.array([[.8,.5], [0,.8]]), jnp.array([[0],[0.8]])
lds = LDS(state_size= B.shape[0], action_size=B.shape[1], A=A, B=B)

In [282]:
bpc = BPC(lds.A, lds.B, lr_scale=1e-3, delta=0.01)
lqr = LQR(A, B)

In [283]:
def get_err(T, lds, controller):
    lds.reset()
    avg_err = 0
    err = 0
    for i in range(T):
        try:
            action = controller(lds.state, err)
        except:
            action = controller(lds.state)
        lds.step(action)
        lds.state += 0.03 * jnp.sin(i) # add sine noise
        err = jnp.linalg.norm(lds.state)+jnp.linalg.norm(action)
        avg_err += err/T
    return avg_err

In [284]:
T = 100

In [285]:
print("BPC incurs ", get_err(T, lds, bpc), " loss")
print("LQR incurs ", get_err(T, lds, lqr), " loss")

BPC incurs  0.07014202  loss
LQR incurs  0.10718855  loss
